<font size="+3"><b>Assignment 4: Pipelines and Hyperparameter Tuning</b></font>

***
* **Full Name** = Justin Pham   
* **UCID** = 30139323
***

<font color='Blue'>
In this assignment, you will be putting together everything you have learned so far. You will need to find your own dataset, do all the appropriate preprocessing, test different supervised learning models, and evaluate the results. More details for each step can be found below. You will also be asked to describe the process by which you came up with the code. More details can be found below. Please cite any websites or AI tools that you used to help you with this assignment.
</font>

<font color='Red'>
For this assignment, in addition to your .ipynb file, please also attach a PDF file. To generate this PDF file, you can use the print function (located under the "File" within Jupyter Notebook). Name this file ENGG444_Assignment##__yourUCID.pdf (this name is similar to your main .ipynb file). We will evaluate your assignment based on the two files and you need to provide both.
</font>


|         **Question**         | **Point(s)** |
|:----------------------------:|:------------:|
|  **1. Preprocessing Tasks**  |              |
|              1.1             |       2      |
|              1.2             |       2      |
|              1.3             |       4      |
| **2. Pipeline and Modeling** |              |
|              2.1             |       3      |
|              2.2             |       6      |
|              2.3             |       5      |
|              2.4             |       3      |
|     **3. Bonus Question**    |     **2**    |
|           **Total**          |    **27**    |

## **0. Dataset**

This data is a subset of the **Heart Disease Dataset**, which contains information about patients with possible coronary artery disease. The data has **14 attributes** and **294 instances**. The attributes include demographic, clinical, and laboratory features, such as age, sex, chest pain type, blood pressure, cholesterol, and electrocardiogram results. The last attribute is the **diagnosis of heart disease**, which is a categorical variable with values from 0 (no presence) to 4 (high presence). The data can be used for **classification** tasks, such as predicting the presence or absence of heart disease based on the other attributes.

In [2]:
import pandas as pd

# Define the data source link
_link = 'https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.hungarian.data'

# Read the CSV file into a Pandas DataFrame, considering '?' as missing values
df = pd.read_csv(_link, na_values='?',
                 names=['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs',
                        'restecg', 'thalach', 'exang', 'oldpeak', 'slope',
                        'ca', 'thal', 'num'])

# Display the DataFrame
display(df)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,28,1,2,130.0,132.0,0.0,2.0,185.0,0.0,0.0,NaN,NaN,NaN,0
1,29,1,2,120.0,243.0,0.0,0.0,160.0,0.0,0.0,NaN,NaN,NaN,0
2,29,1,2,140.0,NaN,0.0,0.0,170.0,0.0,0.0,NaN,NaN,NaN,0
3,30,0,1,170.0,237.0,0.0,1.0,170.0,0.0,0.0,NaN,NaN,6.0,0
4,31,0,2,100.0,219.0,0.0,1.0,150.0,0.0,0.0,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,52,1,4,160.0,331.0,0.0,0.0,94.0,1.0,2.5,NaN,NaN,NaN,1
290,54,0,3,130.0,294.0,0.0,1.0,100.0,1.0,0.0,2.0,NaN,NaN,1
291,56,1,4,155.0,342.0,1.0,0.0,150.0,1.0,3.0,2.0,NaN,NaN,1
292,58,0,2,180.0,393.0,0.0,0.0,110.0,1.0,1.0,2.0,NaN,7.0,1


# **1. Preprocessing Tasks**

- **1.1** Find out which columns have more than 60% of their values missing and drop them from the data frame. Explain why this is a reasonable way to handle these columns. **(2 Points)**

- **1.2** For the remaining columns that have some missing values, choose an appropriate imputation method to fill them in. You can use the `SimpleImputer` class from `sklearn.impute` or any other method you prefer. Explain why you chose this method and how it affects the data. **(2 Points)**

- **1.3** Assign the `num` column to the variable `y` and the rest of the columns to the variable `X`. The `num` column indicates the presence or absence of heart disease based on the angiographic disease status of the patients. Create a `ColumnTransformer` object that applies different preprocessing steps to different subsets of features. Use `StandardScaler` for the numerical features, `OneHotEncoder` for the categorical features, and `passthrough` for the binary features. List the names of the features that belong to each group and explain why they need different transformations. You will use this `ColumnTransformer` in a pipeline in the next question. **(4 Points)**

<font color='Green'><b>Answer:</b></font>

- **1.1** missing data is generally unfavourable in machine learning since it can result in more inaccurate models and more difficult to tune and train.

In [3]:
# 1.1
# Add necessary code here.
# TO DO: Check if there are any missing values and fill them in if necessary
threshHold = 0.6 * len(df)
df.dropna(thresh=threshHold, axis=1, inplace=True)

print(df.isna().sum())
# print number of unique values
print(df.nunique())

age          0
sex          0
cp           0
trestbps     1
chol        23
fbs          8
restecg      1
thalach      1
exang        1
oldpeak      0
num          0
dtype: int64
age          38
sex           2
cp            4
trestbps     31
chol        153
fbs           2
restecg       3
thalach      71
exang         2
oldpeak      10
num           2
dtype: int64


<font color='Green'><b>Answer:</b></font>

- **1.2** I chose simpleImputer because it is fast and easy to use. I am only filling in missing data with the mean of that column. Because we filled in missing data with the mean, this may we result in our model having slightly less accuracy.

In [4]:
# 1.2
# Add necessary code here.
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
print(df.isna().sum())

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
num         0
dtype: int64


<font color='Green'><b>Answer:</b></font>

- **1.3** .....................

In [5]:
# 1.3
# Add necessary code here.
y = df['num']
X = df.drop(columns='num')

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
# get features from X where column is numerical
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
categorial_features = X.select_dtypes(include=['object']).columns
# Binary features is the features in numerical features that have only 2 unique values
binary_features = X[numerical_features].nunique()[X[numerical_features].nunique() == 2].index
# Drop the binary features from numerical_features
numerical_features = numerical_features.drop(binary_features)

ss = ColumnTransformer(
    transformers=[("ss", StandardScaler(), numerical_features)],
)
ohe = ColumnTransformer(
    transformers=[("ohe", OneHotEncoder(), categorial_features)]
)
pt = ColumnTransformer(
    transformers=[("pt", "passthrough", binary_features)]
)
print("numerical_features: ", numerical_features)
print("categorial_features: ", categorial_features)
print("binary_features: ", binary_features)
X.nunique()

numerical_features:  Index(['age', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang',
       'oldpeak'],
      dtype='object')
categorial_features:  Index([], dtype='object')
binary_features:  Index(['sex'], dtype='object')


age          38
sex           2
cp            4
trestbps     32
chol        154
fbs           3
restecg       4
thalach      72
exang         3
oldpeak      10
dtype: int64

# **2. Pipeline and Modeling**

- **2.1** Create **three** `Pipeline` objects that take the column transformer from the previous question as the first step and add one or more models as the subsequent steps. You can use any models from `sklearn` or other libraries that are suitable for binary classification. For each pipeline, explain **why** you selected the model(s) and what are their **strengths and weaknesses** for this data set. **(3 Points)**

- **2.2** Use `GridSearchCV` to perform a grid search over the hyperparameters of each pipeline and find the best combination that maximizes the cross-validation score. Report the best parameters and the best score for each pipeline. Then, update the hyperparameters of each pipeline using the best parameters from the grid search. **(6 Points)**

- **2.3** Form a stacking classifier that uses the three pipelines from the previous question as the base estimators and a meta-model as the `final_estimator`. You can choose any model for the meta-model that is suitable for binary classification. Explain **why** you chose the meta-model and how it combines the predictions of the base estimators. Then, use `StratifiedKFold` to perform a cross-validation on the stacking classifier and present the accuracy scores and F1 scores for each fold. Report the mean and the standard deviation of each score in the format of `mean ± std`. For example, `0.85 ± 0.05`. Interpret the results and compare them with the baseline scores from the previous assignment. **(5 Points)**

- **2.4**: Interpret the final results of the stacking classifier and compare its performance with the individual models. Explain how stacking classifier has improved or deteriorated the prediction accuracy and F1 score, and what are the possible reasons for that. **(3 Points)**

<font color='Green'><b>Answer:</b></font>

- **2.1** 

The three modles I have chosen are decision tree classifier, support vector classifier, and KNN classifier.


I chose decision tree classifier because decision trees are great options for classification and regression models. This makes them flexible for different types of data.

Strengths:
1. Works for both classficiation and regression, making them very flexible
1. Can be combined in forests for even greater accuracy and performance.
1. Easy to interpret and visualize, making it clear how decisions are made.

Weaknesses:
1. Prone to overfitting if not properly tuned.
1. Not very powerful on it's own, usually combined with forests.
1. Sensitive to changes in the training data, potentially leading to unstable predictions for small variations.

I chose support vector classifier because SVC can capture relationships between different features.

Strengths:
1. Powerful for high-dimensional data and works well with small datasets.
1. Can handle complex non-linear relationships between features using kernel functions.

Weaknesses:
1. Hard to explain and understand
1. Can be computationally expensive for very large datasets due to the optimization process.
1. Choosing the right kernel function and its hyperparameters can be crucial for performance and can involve trial-and-error.


I chose knn classifier because it is simple to explain and understand.

Strengths:
1. Easy to explain and understand
1. Makes no assumptions about the underlying data distribution.
1. Minimal tuning necessary


Weaknesses:
1. Performance can be slow for high-dimensional data due to distance calculations for all neighbors.
1. Can suffer from the "curse of dimensionality" in high dimensions, where distances become meaningless.


In [12]:
# 2.1
# Add necessary code here.
from sklearn.pipeline import Pipeline
from sklearn import tree, svm
from sklearn.neighbors import KNeighborsClassifier


# pipe_dt = Pipeline( [('ss', ss), ('ohe', ohe), ("pt", pt), ("dt", tree.DecisionTreeClassifier())] )
pipe_dt = Pipeline( [('scalar', StandardScaler()), ("classifier", tree.DecisionTreeClassifier())] )
# pipe_svm = Pipeline([('ss', ss), ('ohe', ohe), ("pt", pt), ("svm", svm.SVC())])
# pipe_knn = Pipeline([('ss', ss), ('ohe', ohe), ("pt", pt), ("knn", KNeighborsClassifier())])

<font color='Green'><b>Answer:</b></font>

- **2.2** .....................

In [ ]:
# 2.2
# Add necessary code here.
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

RANDOM_STATE = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=RANDOM_STATE)

param_grid_dt = {
    "dt__max_depth": [None, 3, 7, 11],
}
param_grid_knn = {
    "n_neighbours": [3, 5, 7, 9, 11],
}
param_grid_svm = {
    "C": [1, 10, 100],
    "gamma": [1, 10, 100],
    "kernel" : ["linear", "poly", "rbf", "sigmoid"]
}

grid_search_dt = GridSearchCV(pipe_dt, param_grid_dt, cv=5, scoring="accuracy", n_jobs=-1)

grid_search_dt.fit(X_train, y_train)
# print(type(X_train))
# print(type(y_train))




<font color='Green'><b>Answer:</b></font>

- **2.3** .....................

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV


rf_pipeline = Pipeline([
    ('ss', StandardScaler()),         # Step 1: Standardize features by removing the mean and scaling to unit variance
    ('classifier', RandomForestClassifier())  # Step 2: Random Forest classifier for classification tasks
])

param_grid_rf = {
    'classifier__n_estimators': [50, 75, 100],  # Step 1: Number of trees in the forest [100, 200, 300]
    'classifier__max_depth': [5, 10, 15]          # Step 2: Maximum depth of the tree  [5, 10, 15]
}

scoring = {
    'accuracy': make_scorer(accuracy_score),         # Scoring based on accuracy_score
    'f1_score': make_scorer(f1_score)                # Scoring based on F1_score
}

grid_search_rf = GridSearchCV(rf_pipeline, param_grid_rf, cv=5, scoring=scoring, refit='f1_score',  n_jobs=-1)


In [ ]:
# 2.3
# Add necessary code here.

<font color='Green'><b>Answer:</b></font>

- **2.4** .....................

**Bonus Question**: The stacking classifier has achieved a high accuracy and F1 score, but there may be still room for improvement. Suggest **two** possible ways to improve the modeling using the stacking classifier, and explain **how** and **why** they could improve the performance. **(2 points)**

<font color='Green'><b>Answer:</b></font>